In [ ]:
from google.colab import drive
#drive.flush_and_unmount()


In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/MyDrive/RCGP/RCGP/RCGP-main/

/content/gdrive/MyDrive/RCGP/RCGP/RCGP-main


In [ ]:
ls-lrt

total 13
-rw------- 1 root root  482 Aug  2 19:19 README.md
-rw------- 1 root root    0 Aug  2 19:19 __init__.py
drwx------ 2 root root 4096 Aug  2 19:20 rcgp/
drwx------ 2 root root 4096 Aug  2 19:20 experiments/
drwx------ 2 root root 4096 Aug  2 19:48 my_dataset/


In [ ]:
import sys
import os

SCRIPT_DIR = os.path.dirname(os.path.abspath(os.getcwd()))
sys.path.append(SCRIPT_DIR)


In [ ]:
import tensorflow as tf
import numpy as np
# import gpflow


In [ ]:
!pip install check_shapes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 9.9 MB/s eta 0:00:00


In [ ]:
import check_shapes

In [ ]:
!pip install gpflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 392.9/392.9 kB 18.2 MB/s eta 0:00:00


In [ ]:
from rcgp.rcgp import RCGPR
from rcgp.w import IMQ

import matplotlib.pyplot as plt

In [ ]:
seed = 10
sigma_n = 0.1

In [ ]:
!pwd

/content/gdrive/MyDrive/RCGP/RCGP/RCGP-main


In [ ]:
import pandas



#get a data frame with selected columns
# FORMAT = ['Col_1', 'Col_2', 'Col_3']
# df_selected = df[FORMAT]

In [ ]:
x = pandas.read_excel('/content/gdrive/MyDrive/RCGP/RCGP/RCGP-main/my_dataset/Neal_data.xls',sheet_name='Sheet1', header=None, usecols=[0]) #, names=['A'])
y = pandas.read_excel('/content/gdrive/MyDrive/RCGP/RCGP/RCGP-main/my_dataset/Neal_data.xls',sheet_name='Sheet1', header=None, usecols=[1]) #, names=['B'])
xt = pandas.read_excel('/content/gdrive/MyDrive/RCGP/RCGP/RCGP-main/my_dataset/Neal_data.xls',sheet_name='Sheet2', header=None, usecols=[0]) #, names=['A'])
yt = pandas.read_excel('/content/gdrive/MyDrive/RCGP/RCGP/RCGP-main/my_dataset/Neal_data.xls',sheet_name='Sheet2', header=None, usecols=[1]) #, names=['B'])

#print the column names
print (x)
print (xt)
print (y)
print (yt)

          0
0  -0.17308
1  -0.47952
2  -0.78357
3  -0.63218
4  -0.91060
..      ...
95 -0.12340
96 -1.13396
97  0.51589
98  0.48192
99  0.32773

[100 rows x 1 columns]
        0
0   -2.70
1   -2.69
2   -2.68
3   -2.67
4   -2.66
..    ...
866  5.96
867  5.97
868  5.98
869  5.99
870  6.00

[871 rows x 1 columns]
          1
0   1.10456
1   0.63622
2   0.16344
3  -0.86175
4   0.06405
..      ...
95  1.15153
96  0.25201
97  1.79423
98  1.87952
99  1.88636

[100 rows x 1 columns]
            1
0   -1.069877
1   -1.057639
2   -1.045090
3   -1.032236
4   -1.019082
..        ...
866  2.526786
867  2.518241
868  2.509842
869  2.501599
870  2.493519

[871 rows x 1 columns]


In [ ]:
print(y.iloc[6, 0])
print(np.arange(4.3, 4.6, 0.1))
print(x.iloc[21,0])

0.74596
[4.3 4.4 4.5]
0.84909


In [ ]:
#Adding v outliers and b outliers
out_idx_v = [6, 7, 8, 9, 10, 14, 60, 69]
out_idx_b = [20, 21, 22]
out_idx_g = [49, 50, 51, 52, 53, 54]

# Adding bad leverage points
# x.iloc[out_idx_b] = [[4.3], [4.6], [0.1]]
# y.iloc[out_idx_b] = [[8.4763], [9.1938], [0.2833]]

# Adding vertical outliers
# y.iloc[out_idx_v] = [[90.5], [8.6], [98.1], [5.3], [5.2], [6.1], [1], [8]]
y.iloc[out_idx_v]=  [[5], [5], [5], [5], [5], [5], [5], [5]]   #5 * np.ones(len(out_idx_v))

# Adding good leverage points (commented out)
# x.iloc[out_idx_g]=[[2.8],[3.82],[3.84],[3.16],[2.88],[2.9]]
# y.iloc[out_idx_g]= [[2.0230],[1.5103], [1.5011], [2.0535],  [2.0688],  [2.0768]]


# Exporting data (commented out)
# np.savetxt('Neal_data.csv', np.column_stack((x, y)), delimiter=',')

# Adding noise
n = len(x)
# noise = np.random.normal(0.01, 0.08, n)  # Uncomment for normal noise
# noise = 0.01 * np.random.standard_t(10, n)  # Uncomment for Student's-t noise with nu=10
# noise = 0.1 * np.random.laplace(0, 1, n)  # Uncomment for Laplacian noise
noise = 0.01 * np.random.standard_t(1, n)  # Uncomment for Student's-t noise with nu=1 (Cauchy)


In [ ]:
y.iloc[:, 0] = y.iloc[:, 0] + noise  # Access the first (and only) column of y
print (y)
print (x)


           1
0   1.102697
1   0.558448
2   0.274540
3  -0.798616
4   0.116976
..       ...
95  1.151094
96  0.258683
97  1.790455
98  1.895370
99  1.871432

[100 rows x 1 columns]
          0
0  -0.17308
1  -0.47952
2  -0.78357
3  -0.63218
4  -0.91060
..      ...
95 -0.12340
96 -1.13396
97  0.51589
98  0.48192
99  0.32773

[100 rows x 1 columns]


In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import gpflow

# from tueplots import bundles

In [ ]:
# Trining standadrf GP
standard_gp = gpflow.models.GPR(
    (x, y),
    kernel=gpflow.kernels.SquaredExponential(lengthscales=0.01,variance = 0.01),
    noise_variance = 0.01
)

opt = gpflow.optimizers.Scipy()
opt_options = dict()
res = opt.minimize(standard_gp.training_loss_closure(), standard_gp.trainable_variables, options=opt_options)
print(res)
# gpflow.utilities.print_summary(standard_gp, "notebook")

  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 163.46306787645662
        x: [ 3.526e-01  1.851e+00  9.987e-01]
      nit: 19
      jac: [ 5.635e-05  5.024e-05 -2.106e-04]
     nfev: 22
     njev: 22
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>


In [ ]:
model = RCGPR(
    (x, y),
    kernel=gpflow.kernels.SquaredExponential(),
    weighting_function=IMQ(1.5),
    noise_variance=standard_gp.likelihood.variance)

gpflow.set_trainable(model.likelihood.variance, False)
opt = gpflow.optimizers.Scipy()
opt_options = dict()
res = opt.minimize(model.training_loss_closure(), model.trainable_variables, options=opt_options)
print(res)
gpflow.utilities.print_summary(model, "notebook")



  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 156.75788652254153
        x: [ 5.931e-01  1.133e+01  3.053e+00]
      nit: 18
      jac: [-2.638e-05 -1.945e-06  1.053e-05]
     nfev: 20
     njev: 20
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>


name,class,transform,prior,trainable,shape,dtype,value
RCGPR.kernel.variance,Parameter,Softplus,,True,(),float64,11.3321
RCGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,1.03303
RCGPR.likelihood.variance,Parameter,Softplus + Shift,,False,(),float64,1.31233
RCGPR.weighting_function.C,Parameter,Softplus,,True,(),float64,3.09947


In [ ]:
import sklearn
from sklearn.preprocessing import StandardScaler, RobustScaler


In [ ]:
!pip install tueplots


In [ ]:
from tueplots import bundles


In [ ]:
# predictions
if isinstance(xt, pd.DataFrame):  # convert xt into tensorflow data types
    xt = xt.values


In [ ]:
Eft, Varft = model.predict_f(xt, full_cov=False)
std_ft = np.sqrt(Varft)

In [ ]:
print (Varft)
print (Eft)

tf.Tensor(
[[ 1.25840796]
 [ 1.23163522]
 [ 1.20557824]
 [ 1.18022487]
 [ 1.15556265]
 [ 1.13157884]
 [ 1.10826044]
 [ 1.0855942 ]
 [ 1.06356666]
 [ 1.04216416]
 [ 1.02137285]
 [ 1.00117874]
 [ 0.98156769]
 [ 0.96252547]
 [ 0.94403773]
 [ 0.92609007]
 [ 0.90866802]
 [ 0.89175708]
 [ 0.87534276]
 [ 0.85941056]
 [ 0.843946  ]
 [ 0.82893467]
 [ 0.81436221]
 [ 0.80021434]
 [ 0.7864769 ]
 [ 0.77313583]
 [ 0.7601772 ]
 [ 0.74758726]
 [ 0.73535241]
 [ 0.72345921]
 [ 0.71189446]
 [ 0.70064513]
 [ 0.68969843]
 [ 0.67904181]
 [ 0.66866297]
 [ 0.65854984]
 [ 0.64869065]
 [ 0.6390739 ]
 [ 0.62968836]
 [ 0.62052312]
 [ 0.61156756]
 [ 0.60281138]
 [ 0.59424457]
 [ 0.58585747]
 [ 0.57764073]
 [ 0.56958535]
 [ 0.56168262]
 [ 0.55392422]
 [ 0.54630213]
 [ 0.53880867]
 [ 0.53143653]
 [ 0.52417869]
 [ 0.51702852]
 [ 0.50997968]
 [ 0.5030262 ]
 [ 0.49616242]
 [ 0.48938301]
 [ 0.48268297]
 [ 0.47605763]
 [ 0.46950261]
 [ 0.46301387]
 [ 0.45658763]
 [ 0.45022046]
 [ 0.44390918]
 [ 0.4376509 ]
 [ 0.43144304]

In [ ]:
def RMSE(Eft, yt):
  """
  Calculates the Root Mean Squared Error (RMSE) between two arrays.

  Args:
    Eft: A numpy array of predicted values.
    yt: A numpy array of true values.

  Returns:
    The calculated RMSE.
  """

  rmse = np.sqrt(np.mean((Eft - yt) ** 2))
  return rmse

In [ ]:
def MAE(Eft, yt):
  """
  Calculates the Mean Absolute Error (MAE) between two arrays.

  Args:
    Eft: A numpy array of predicted values.
    yt: A numpy array of true values.

  Returns:
    The calculated MAE.
  """

  mae = np.mean(np.abs(Eft - yt))
  return mae

In [ ]:
def NLP(means, variances, yTrue):
  """
  Calculates NLP

  Args:
    means: A numpy array of means.
    variances: A numpy array of variances.
    yTrue: A numpy array of true values.

  Returns:
    The calculated value.
  """

  value = 0.5 * np.mean(np.log(2 * np.pi * variances) + ((means - yTrue) ** 2) / variances)
  return value

In [ ]:
Results = np.zeros((1, 3))
Results[0,0]=RMSE(Eft, yt)
Results[0,1]=MAE(Eft, yt)
Results[0,2]=NLP(Eft,Varft, yt)
print(Results)

[[1.09783913 0.76432326 1.02358648]]
